In [5]:
import pandas as pd
import numpy as np

# Function to calculate RMSSD and SDNN
def calculate_hrv_metrics(ibi_data):
    valid_ibi = pd.Series(ibi_data)
    diff_nn_intervals = np.diff(valid_ibi)
    squared_diffs = np.square(diff_nn_intervals)
    rmssd = np.sqrt(np.mean(squared_diffs))
    sdnn = np.std(valid_ibi, ddof=1)
    return rmssd, sdnn

# Load IBI data for all three test sessions and the baseline
ibi_baseline = pd.read_csv('/Users/urmebose/Desktop/People/Nuro/ibi.csv')
ibi_01 = pd.read_csv('/Users/urmebose/Desktop/People/Nuro/ibi_01.csv')
ibi_02 = pd.read_csv('/Users/urmebose/Desktop/People/Nuro/ibi_02.csv')
ibi_03 = pd.read_csv('/Users/urmebose/Desktop/People/Nuro/ibi_03.csv')

# Ensure valid IBI values for baseline
ibi_baseline = ibi_baseline[ibi_baseline['ibi'] > 0]

# Calculate baseline HRV metrics
rmssd_baseline, sdnn_baseline = calculate_hrv_metrics(ibi_baseline['ibi'])

# Load psychometric test data for all three sessions
psychometric_01 = pd.read_csv('/Users/urmebose/Desktop/People/Nuro/Psychometric_Test_Results_01.csv')
psychometric_02 = pd.read_csv('/Users/urmebose/Desktop/People/Nuro/Psychometric_Test_Results_02.csv')
psychometric_03 = pd.read_csv('/Users/urmebose/Desktop/People/Nuro/Psychometric_Test_Results_03.csv')

# Clean and prepare the psychometric data
psychometric_01.columns = ['Type', 'Test', 'Question', 'Answer', 'Time(s)', 'Question Start Time', 'Question Answer Time']
psychometric_02.columns = ['Type', 'Test', 'Question', 'Answer', 'Time(s)', 'Question Start Time', 'Question Answer Time']
psychometric_03.columns = ['Type', 'Test', 'Question', 'Answer', 'Time(s)', 'Question Start Time', 'Question Answer Time']

psychometric_01['Question Start Time'] = pd.to_datetime(psychometric_01['Question Start Time'], errors='coerce').dt.tz_localize(None)
psychometric_02['Question Start Time'] = pd.to_datetime(psychometric_02['Question Start Time'], errors='coerce').dt.tz_localize(None)
psychometric_03['Question Start Time'] = pd.to_datetime(psychometric_03['Question Start Time'], errors='coerce').dt.tz_localize(None)
psychometric_01['Question Answer Time'] = pd.to_datetime(psychometric_01['Question Answer Time'], errors='coerce').dt.tz_localize(None)
psychometric_02['Question Answer Time'] = pd.to_datetime(psychometric_02['Question Answer Time'], errors='coerce').dt.tz_localize(None)
psychometric_03['Question Answer Time'] = pd.to_datetime(psychometric_03['Question Answer Time'], errors='coerce').dt.tz_localize(None)

psychometric_01 = psychometric_01.dropna(subset=['Question Start Time'])
psychometric_02 = psychometric_02.dropna(subset=['Question Start Time'])
psychometric_03 = psychometric_03.dropna(subset=['Question Start Time'])

# Filter for relevant questions and ensure correct counts
types_count = {
    'HADS': 14,
    'STAI-S': 20,
    'STAI-T': 20,
    'BFI': 10,
    'FQ': 24
}

def filter_correct_questions(df, types_count):
    filtered_df = pd.DataFrame()
    for q_type, count in types_count.items():
        filtered_df = pd.concat([filtered_df, df[df['Type'] == q_type].head(count)])
    return filtered_df

questions_01 = filter_correct_questions(psychometric_01, types_count)
questions_02 = filter_correct_questions(psychometric_02, types_count)
questions_03 = filter_correct_questions(psychometric_03, types_count)

# Function to get IBI data for questions
def get_ibi_data(question, ibi_data):
    ibi_data['datetime'] = pd.to_datetime(ibi_data['datetime'], errors='coerce').dt.tz_localize(None)
    ibi_data_filtered = ibi_data[
        (ibi_data['datetime'] >= question['Question Start Time']) &
        (ibi_data['datetime'] <= question['Question Answer Time'])
    ]
    return ibi_data_filtered['ibi'].values

# Function to detect significant increases compared to baseline
def detect_significant_increase(test_metrics, baseline_metrics):
    rmssd_increase = test_metrics[0] > baseline_metrics[0]
    sdnn_increase = test_metrics[1] > baseline_metrics[1]
    return rmssd_increase, sdnn_increase

# Calculate HRV metrics for each question
def calculate_hrv_metrics_for_questions(questions, ibi_data, baseline_metrics):
    results = []
    for _, question in questions.iterrows():
        ibi_values = get_ibi_data(question, ibi_data)
        ibi_values = ibi_values[ibi_values > 0]  # Ensure valid IBI values
        if len(ibi_values) > 1:  # Ensure there are enough IBI values to calculate metrics
            rmssd, sdnn = calculate_hrv_metrics(ibi_values)
            significant_increases = detect_significant_increase((rmssd, sdnn), baseline_metrics)
            results.append({
                'Type': question['Type'],
                'Start Time': question['Question Start Time'],
                'End Time': question['Question Answer Time'],
                'Score': question['Answer'],
                'RMSSD': round(rmssd, 2),
                'SDNN': round(sdnn, 2),
                'RMSSD Increase': 'Yes' if significant_increases[0] else 'No',
                'SDNN Increase': 'Yes' if significant_increases[1] else 'No'
            })
    return pd.DataFrame(results)

# Calculate HRV metrics for each test
baseline_metrics = (rmssd_baseline, sdnn_baseline)
results_01 = calculate_hrv_metrics_for_questions(questions_01, ibi_01, baseline_metrics)
results_02 = calculate_hrv_metrics_for_questions(questions_02, ibi_02, baseline_metrics)
results_03 = calculate_hrv_metrics_for_questions(questions_03, ibi_03, baseline_metrics)

# Combine results into a single dataframe for better presentation
results_01['Test'] = 'Test 01'
results_02['Test'] = 'Test 02'
results_03['Test'] = 'Test 03'

combined_results = pd.concat([results_01, results_02, results_03], ignore_index=True)

# Reorder columns to place 'Test' as the first column
combined_results = combined_results[['Test', 'Type', 'Start Time', 'End Time', 'Score', 'RMSSD', 'SDNN', 'RMSSD Increase', 'SDNN Increase']]

# Save the combined results to a new CSV file
combined_results.to_csv('/Users/urmebose/Desktop/People/Nuro/QQHRV.csv', index=False)

combined_results.head()  # Display the first few rows of the combined results for verification

,Test,Type,Start Time,End Time,Score,RMSSD,SDNN,RMSSD Increase,SDNN Increase
0,Test 01,HADS,2024-06-13 11:30:18.058,2024-06-13 11:30:30.793,1,72.54,66.81,Yes,Yes
1,Test 01,HADS,2024-06-13 11:30:30.793,2024-06-13 11:30:38.713,1,71.46,67.38,Yes,Yes
2,Test 01,HADS,2024-06-13 11:30:38.713,2024-06-13 11:30:46.577,2,33.85,22.36,No,No
3,Test 01,HADS,2024-06-13 11:30:46.577,2024-06-13 11:30:54.409,1,34.47,24.25,No,No
4,Test 01,HADS,2024-06-13 11:30:54.409,2024-06-13 11:30:57.497,1,10.86,7.53,No,No
